In [29]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(os.path.abspath('../../AB-Generic-Python-Toolbox/'))
import numpy as np
from pathlib import Path
import numpy as np
import pyaudio as pa
from scipy.signal import resample
from scipy.io.wavfile import read as wavread

# load the toolbox package
from GpyT.Demo.proceduralDemo import demo4_procedural

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Notes
* The only I/O constrainst are really around the vocoder
* These include charge balance (all electrodes integrate to zero over time)
* Also include current thresholds

* The submissions are the electrodagram matricies, not the audio from the vocoder

# strategy for continuous ML loop
!['figure'](figures/results.png)

# Run Demo

In [41]:

results  = demo4_procedural()

Channels [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15] are very similar to the default output.


In [42]:
wavIn = wavread(results['sourceName'])
wavData = wavIn[1]/(2**15-1)
wavFs = wavIn[0]

wavResampled = resample(wavData,((results['audioFs']/wavFs)*wavData.shape[0]).astype(int))
input1 = np.float32(np.concatenate((wavResampled,np.zeros(results['audioFs']))))

output1 = np.float32(np.concatenate((np.zeros(results['audioFs']),results['audioOut'])))
p = pa.PyAudio()
devInfo = p.get_default_output_device_info()
devIndex = devInfo['index']
nChan = 1

stream = p.open(format=pa.paFloat32,
channels=nChan,
rate=results['audioFs'],
output=True,
output_device_index = devIndex
)

inData = input1.astype(np.float32).tobytes()
outData1 = output1.astype(np.float32).tobytes()


stream.write(outData1)


stream.write(inData)
stream.close()
    

# testing on a different file

In [35]:
example_file=os.path.abspath('../sample_data/sentence1_55_clean.wav')

In [36]:
results  = demo4_procedural(example_file)

In [37]:
wavIn = wavread(results['sourceName'])
wavData = wavIn[1]/(2**15-1)
wavFs = wavIn[0]

wavResampled = resample(wavData,((results['audioFs']/wavFs)*wavData.shape[0]).astype(int))
input1 = np.float32(np.concatenate((wavResampled,np.zeros(results['audioFs']))))

output1 = np.float32(np.concatenate((np.zeros(results['audioFs']),results['audioOut'])))
p = pa.PyAudio()
devInfo = p.get_default_output_device_info()
devIndex = devInfo['index']
nChan = 1

stream = p.open(format=pa.paFloat32,
channels=nChan,
rate=results['audioFs'],
output=True,
output_device_index = devIndex
)

inData = input1.astype(np.float32).tobytes()
outData1 = output1.astype(np.float32).tobytes()


stream.write(outData1)


stream.write(inData)
stream.close()